In [1]:
import pandas as pd
import numpy as np
import math as m
import DataHandling

In [12]:
roadId = input('Please write the Road ID: ')

Please write the Road ID: N1


In [22]:
roads = DataHandling.readRoads(roadId)
bridges = DataHandling.readBridges(roadId)
widths = DataHandling.readWidths(roadId)
traffic = DataHandling.readTraffic(roadId)
roadnames = DataHandling.readBridges('').drop_duplicates('road', keep = 'first')['road'].tolist()

In [23]:
def formatWidthData(dataframe = widths):
    dataframe = dataframe.drop('width', 1)
    dataframe = dataframe.drop('roadId', 1)
    dataframe = dataframe.sort_values(by=['road','startChainage']).reset_index()
    dataframe = dataframe.drop('index', 1)
    return dataframe

In [24]:
def formatBridgeData(dataframe = bridges):
    dataframenoduplicates = dataframe.drop_duplicates('LRPName', keep = 'first')
    dataframemodified = dataframenoduplicates.ix[:,:10]
    dataframemodified = dataframemodified.drop('km', 1)
    dataframemodified = dataframemodified.drop('structureNr', 1)
    dataframemodified = dataframemodified.drop('type', 1)
    dataframemodified = dataframemodified.drop('roadName', 1)
    dataframemodified = dataframemodified.drop('length', 1)
    dataframemodified = dataframemodified.drop('name', 1)
    dataframemodified = dataframemodified.sort_values(by=['road','chainage']).reset_index()
    dataframemodified = dataframemodified.drop('index', 1)
    return dataframemodified

In [25]:
def formatTrafficData(dataframe = traffic, sideofroad = 'both'):
    dataframe = dataframe.drop('name',1)
    dataframe = dataframe.drop('start_lrp',1)
    dataframe = dataframe.drop('start_offset',1)
    dataframe = dataframe.drop('end_lrp',1)
    dataframe = dataframe.drop('end_offset',1)
    dataframe = dataframe.drop('length',1)
    dataframe = dataframe.reset_index()
    dataframe = dataframe.drop('index',1)
    dataframe.columns = dataframe.columns.str.replace('start_chainage','startChainage')
    dataframe.columns = dataframe.columns.str.replace('end_chainage','endChainage')
    dataframe = dataframe.sort_values(by = ['road','startChainage','linkNo'] ).reset_index()
    dataframe = dataframe.drop('index', 1)
    if sideofroad == 'both':
        dataframe = dataframe.groupby(['road','startChainage'],as_index=False).sum()
    return dataframe

In [26]:
def mergeFiles(trafficData=formatTrafficData(), bridgeData = formatBridgeData(), widthData = formatWidthData(), roadside = 'both'):
    indexesMergedData = widthData['startChainage'].searchsorted(bridgeData['chainage'], side = 'right')
    indexesMergedData =indexesMergedData.tolist()
    addedNrLanes = bridgeData
    list_a = []
    for i in indexesMergedData:
        list_a.append(widthData['nrLanes'][int(i)-1])
    addedNrLanes['nrLanes']=list_a
    bridgeData = addedNrLanes
    addedTrafficData = bridgeData
    for title in trafficData.ix[:,3:].columns:
        indexesMergedData = trafficData['startChainage'].searchsorted(bridgeData['chainage'], side = 'right')
        indexesMergedData =indexesMergedData.tolist()
        list_b=[]
        for i in indexesMergedData:
            list_b.append(trafficData[title][int(i)-1])
        addedTrafficData[title]=list_b
    return addedTrafficData

In [27]:
def calculateLaneTrafficDensity(dataframe = mergeFiles()):
    for title in dataframe.ix[:,5:].columns:
        dataframe[title] = dataframe[title]/dataframe['nrLanes']
    return dataframe

In [28]:
#Truck capacity is max capacity of type of trucks in tonnes
def calculateVulnerability(dataframe = calculateLaneTrafficDensity(), truckcapacity = [15,10,5], socialcarcapacity = [45,30,15,6,4,2,2,1,2,1], conditionlikelihood = [0.05,0.10,0.15,0.20]):
    vulbasematrix = dataframe.ix[:,:4]
    vulbasematrix.loc[vulbasematrix.condition == 'A', 'BridgeFailureLikelihood'] = conditionlikelihood[0]
    vulbasematrix.loc[vulbasematrix.condition == 'B', 'BridgeFailureLikelihood'] = conditionlikelihood[1]
    vulbasematrix.loc[vulbasematrix.condition == 'C', 'BridgeFailureLikelihood'] = conditionlikelihood[2]
    vulbasematrix.loc[vulbasematrix.condition == 'D', 'BridgeFailureLikelihood'] = conditionlikelihood[3]
    cargotitles = ['heavyTruck', 'mediumTruck', 'smallTruck']
    socialtitles = ['largeBus','mediumBus','microBus','utility','car','autoRickshaw','motorcycle','bicycle','cycleRickshaw','cart']
    vulbasematrix['TrafficEconomicVulnerability'] = 0
    for i in range(len(cargotitles)):
        vulbasematrix['TrafficEconomicVulnerability'] =  vulbasematrix['TrafficEconomicVulnerability']+dataframe[cargotitles[i]]*truckcapacity[i]
    vulbasematrix['TotalEconomicVulnerability'] = vulbasematrix['TrafficEconomicVulnerability']*vulbasematrix['BridgeFailureLikelihood']
    vulbasematrix['TrafficSocialVulnerability'] = 0
    for i in range(len(socialtitles)):
        vulbasematrix['TrafficSocialVulnerability'] =  vulbasematrix['TrafficSocialVulnerability']+dataframe[socialtitles[i]]*socialcarcapacity[i]
    vulbasematrix['TotalSocialVulnerability'] = vulbasematrix['TrafficSocialVulnerability']*vulbasematrix['BridgeFailureLikelihood']
    return vulbasematrix

In [29]:
def printRoadResults(dataframe = calculateVulnerability(),  roadId = roadId, root = 'vulnerability'):
    dataframe.to_csv(root+roadId+'.csv')


In [34]:
def calculatevulnerabilitysegment(dataframe = calculateVulnerability(), groupLevel=int(len(calculateVulnerability())/100)):
    global bridges
    londictionary = dict(zip(bridges['LRPName'], bridges['lon']))
    latdictionary = dict(zip(bridges['LRPName'], bridges['lat']))                    
    result = dataframe.groupby(['road',np.floor(dataframe.index/groupLevel)], as_index = False)
    bridgereliability = result.apply(lambda x: np.product(1-x['BridgeFailureLikelihood'])).reset_index(drop=True)
    lista = []
    for i in bridgereliability:
        lista.append(i.max())
    result = result.max()
    result['BridgeFailureLikelihood'] = 1-pd.DataFrame(lista)
    result['TotalSocialVulnerability'] = result['TrafficSocialVulnerability']*result['BridgeFailureLikelihood']
    result['TotalEconomicVulnerability'] = result['TrafficEconomicVulnerability']*result['BridgeFailureLikelihood']
    result = result.drop('condition',1)
    result['Latitude']=result['LRPName']
    result['Longitude']=result['LRPName']
    result = result.replace({"Latitude": latdictionary})
    result = result.replace({"Longitude": londictionary})
    return  result

In [36]:
printRoadResults(calculatevulnerabilitysegment(), root = 'vulnerabilitypersegment')